In [3]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import interpolate

In [24]:
#Functions for examples
def f1():
    f = lambda x: np.cos(x)
    F = lambda x: -sin(x)
    return f, '-sin(x)', F

def f2():
    f = lambda x: np.arctan(x)
    F = lambda x: 1.0/(1.0+x**(2.0))
    return f, '1.0/(1.0+x**(2.0))', F

def f3():
    f = lambda x: 2*x**3
    F = lambda x: (1.0/2.0)*x**(4.0)
    return f, '(1.0/2.0)*x**(4.0)', F

In [42]:
#Recursive variable step size integrator
#Define integrator using simpson's rule; I did this in pset1 for my integrator
def simpson(f, a, b):
    #Splitting into n subintervals
    if a == b:
        return 0
    if a < b:
        sign = 1
    else:
        sign = -1

    h = (a+b)/2 #Get the height
    fh = f(h) #Get the function value at the given height
    fa = f(a)
    fb = f(b)
    dx = (b-a)/6 #Set n=2, so 3*2
    #Integrate with simpson's rule
    i = dx*sign*(fa+4*fh+fb)
    
    return i, h, fh

#Keep iterating over guesses to allow convergence
def dynamic_simpson(f, a, b, h, fa, fb, fh, guess, tolerance, nmax, count=1):
    if count < nmax:
        guess_ah, left_h, left_fh = simpson(f, a, h)
        guess_bh, right_h, right_fh = simpson(f, h, b)
        err = np.abs(guess_ah+guess_bh-guess)
        
        if err < 15*tolerance:
            return guess, count
        else:
            #Provide smaller and smaller tolerances each time, and update the count
            #Separate the left and right side, up until h
            guess_left, count_left = dynamic_simpson(f, a, h, left_h, fa, fh, left_fh, guess_ah, tolerance/2, nmax, count+1)
            guess_right, count_right = dynamic_simpson(f, h, b, right_h, fh, fb, right_fh, guess_bh, tolerance/2, nmax, count+1)
            
            #Combine the two sides for total guess and count
            guess_t = guess_left+guess_right
            count_t = count_left+count_right
            
            return guess_t, count_t
        
    else:
        print('Does not converge. Increase tolerance or nmax')
        exit(1)
        
def summing(f, a, b, tolerance, nmax):
    i, h, fh = simpson(f, a, b)
    result= np.array(dynamic_simpson(f, a, b, h, f(a), f(b), fh, i, tolerance, nmax))
    #Add up the counts and integral for each interval of simpson
    count = np.sum(result[1::2])
    integ = np.sum(result[0::2])
    
    return integ, count

def integ(fun, x):
    f, st, F = fun()
    integral_simpson, c = summing(f, x[0], x[1], 1e-7, 1000)
    print('The integral of', st, 'from', x[0], 'to', x[1], 'is', integral_simpson)
    print('There are', c, 'function calls \n')

In [43]:
#Integrate the example functions
integ(f1, [-1,1])
integ(f2, [-10,10])
integ(f3, [-2,10])

The integral of -sin(x) from -1 to 1 is 1.6829421123469221
There are 80.0 function calls 

The integral of 1.0/(1.0+x**(2.0)) from -10 to 10 is 0.0
There are 1.0 function calls 

The integral of (1.0/2.0)*x**(4.0) from -2 to 10 is 4992.0
There are 1.0 function calls 



The lazy way of integration takes 2*(n+1) function calls, where n is the number of function calls from the step integrator method. For the first example with cos(x), we can see that the lazy integrator would take 162 calls. The arctan(x) and cubic function example take one call to compute the integral; the lazy method would require 4 calls.